## LSTM

In [1]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import time

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


#### validation set

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


#### Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0]) # string => ASCII / fist letter => reorder to ndex of ASCII code '0'

# if input is char type           => ASCII code index
# if          blank or unexpected => 0
def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

# if input is dictionary key           => corresponding ASCII character
# if          not dictionary keyblank or unexpectec => blank
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1) #  reorder to index of ASCII code '0'
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


#### Function to generate a training batch for LSTM model

In [26]:
batch_size = 64
num_unrollings = 10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        
        segment = self._text_size // batch_size # number of batch
        self._cursor = [ offset * segment for offset in range(batch_size)]
        # [0 * seg, 1 * seg, 2 * seg, ... ,  (batch_size - 1) * seg]
        
        self._last_batch = self._next_batch()
        
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data"""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype = np.float)
        # shape = batch_size x vocabulary size
        
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b] # current position of cursor
                                       ])] = 1.0        # position where new cursor starts => 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    
    def next(self):
        """Generate the next array of batches from the data.
        The array consists of the last batch of the previous array, folllowed by num_unrollings new ones."""
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible characters
    back into its (most likely) character representation"""
    return [id2char(c) for c in np.argmax(probabilities, 1)]
    
def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string representation"""
    s = [''] * batches[0].shape[0] # '' 으로 가득찬 batch-size 크기의 리스트
    for b in batches:
        s = [''.join(x) # 모든 characters 의 결과들(문자)을 하나로 연결하여 출력 => string
             for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()), '\n')
print(batches2string(train_batches.next()), '\n')
print(batches2string(valid_batches.next()), '\n')
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad'] 

['ists advoca', 'ary governm', 'hes n

In [7]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized probabilites."""
    
    r = random.uniform(0, 1)
    s = 0
    
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r: # r = 0
            return i
    return len(distribution) - 1 # distribution 의 모든 원소의 합이 r보다 작을 경우

def sample(prediction):
    """Turn a (column prediction) into 1-hot encoded samples"""
    
    p  = np.zeros(shape = [1, vocabulary_size], dtype = np.float)
    p[0, sample_distribution(prediction[0])] = 1.0 # 첫 번째 원소는 1로 고정
    return p

def random_distribution():
    """Generate a random column of probabilities"""
    
    b = np.random.uniform(0.0, 1.0, size = [1, vocabulary_size])
    return b / np.sum(b,1)[:, None]

#### Simple LSTM Model

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    
    # 일반적인 표기법과 달리
    # i => x,
    # h => ouput
    # c=> state
    
    
    # Parameters:
    # Input gate: input, previous ouput, and bias
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Forget gate: input, previous output, and bias
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Memory cell: input, state, and bias
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Output gate: input, previous output, and bias
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Variables saving state across unrollings
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable = False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable = False)
    
    # Classifier weights and biases
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    
    """
    LSTM ARCHITECTURES
    
    i_t       = sigmoid (W_i(h_t-1, x_t) + b_i)
    f_t       = sigmoid (W_f(h_t-1, x_t) + b_f)
    o_t       = sigmoid (W_o(h_t-1, x_t) + b_o)
    
    
    c_tilda_t = tanh    (Wi(h_t-1, x_t)  + b_c)
    
    c_t       = f_t * c_t-1    +    i_t * c_tilda_t
    
    
    h_t       = o_t * tanh(c_t)

    """
    
    
    def lstm_cell(i, o, state):
        """Create a LSTM cell"""
    
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        
        return output_gate * tf.tanh(state), state
    
    # Input data
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.float32, shape = [batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]
    
    # Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    # State saving across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
    
        """
        with g.control_dependencies([a, b, c]):
              `d` and `e` will only run after `a`, `b`, and `c` have executed.
          d = ...
          e = ...
        """
    
        # Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels))
            )
        
     
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    
    # t_list[i] * clip_norm / max(global_norm, clip_norm)
    # where:
    # global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
    
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step = global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling
    sample_input = tf.placeholder(tf.float32, shape = [1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
        )
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
        
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                 saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
# tf.concat?
# tf.nn.xw_plus_b?
# np.concatenate?

In [10]:
num_steps = 7001
summary_frequency = 100

tic = time.clock()

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('initilized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        
        _, batch_loss, predictions, batch_learning_rate = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict = feed_dict
        )
        mean_loss += batch_loss
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                #the mean los is an estimate of the loss over the last few batches
            print(
                "Average loss at step %d: %f learning rate: %.2f" % (step, mean_loss, batch_learning_rate)
            )
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minimbatch perplexity: %2f' % float(np.exp(logprob(predictions, labels))))
            
            if step % (summary_frequency * 10) == 0:
                # Generate some samples
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            
            # Measure validation set perplexity
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
            
toc  = time.clock()

print(toc-tic)

initilized
Average loss at step 0: 3.296609 learning rate: 10.00
Minimbatch perplexity: 27.020841
shoh e hut  pyeetwhgmqdl icatse e zjdateeh   as yr yyiie rasaiq e aunnt v  jdi a
j gcsjqmbzsmqecdepa gnukhexfqdwssjvrynvapha sanvd wrei bcigdr uovvfhbxwfcmiafgtm
xoadef uo llc b g aitv mvudo x rsr izdawdz xeqeervhldhccysyuoacanzwg aifglrhprtr
sctvjiwez wqwlhacauodyik vpl riotibanpkinakjh tahrnqtwyher pweresyjl vscaskdpgmh
kiteaohrhteseyxhkpmckjre qquvyt iyfm  rsoif hti yhlnebbeah ithnn f mpto eomyeu m
Validation set perplexity: 20.09
Average loss at step 100: 2.588399 learning rate: 10.00
Minimbatch perplexity: 10.920722
Validation set perplexity: 10.24
Average loss at step 200: 2.245788 learning rate: 10.00
Minimbatch perplexity: 8.488512
Validation set perplexity: 8.55
Average loss at step 300: 2.096273 learning rate: 10.00
Minimbatch perplexity: 7.439450
Validation set perplexity: 7.91
Average loss at step 400: 1.999881 learning rate: 10.00
Minimbatch perplexity: 7.378782
Validation set

### Problem 1

#### LSTM with single, concatenated matrix

In [11]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    
    
    # 일반적인 표기법과 달리
    # i => x,
    # h => ouput
    # c=> state
       
    
    # Parameters:
    # Input gate: input, previous ouput, and bias
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Forget gate: input, previous output, and bias
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Memory cell: input, state, and bias
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Output gate: input, previous output, and bias
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Concantate parameters into single matrix
    conc_x = tf.concat(1, [ix, fx, cx, ox]) # => conc_x = [ix, fx, cx, ox]
    conc_m = tf.concat(1, [im, fm, cm, om]) # => conc_m = [im, fm, cm, om]
    conc_b = tf.concat(1, [ib, fb, cb, ob]) # => conc_b = [ib, fb, cb, ob]
    
    # Variables saving state across unrollings
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable = False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable = False)
    
    # Classifier weights and biases
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    
    """
    LSTM ARCHITECTURES
    
    i_t       = sigmoid (W_i(h_t-1, x_t) + b_i)
    f_t       = sigmoid (W_f(h_t-1, x_t) + b_f)
    o_t       = sigmoid (W_o(h_t-1, x_t) + b_o)
    
    
    c_tilda_t = tanh    (Wi(h_t-1, x_t)  + b_c)
    
    c_t       = f_t * c_t-1    +    i_t * c_tilda_t
    
    
    h_t       = o_t * tanh(c_t)

    """
    
    
    def lstm_cell(i, o, state):
        """Create a LSTM cell"""
        
        conc_mat = tf.matmul(i, conc_x) + tf.matmul(o, conc_m) + conc_b
        conc_input, conc_forget, update, conc_output = tf.split(1,4, conc_mat)
        
        input_gate = tf.sigmoid(conc_input)
        forget_gate = tf.sigmoid(conc_forget)
        output_gate = tf.sigmoid(conc_output)
        

        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    # Input data
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.float32, shape = [batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]
    
    # Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    
    # State saving across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
    
        """
        with g.control_dependencies([a, b, c]):
              `d` and `e` will only run after `a`, `b`, and `c` have executed.
          d = ...
          e = ...
        """
    
        # Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels))
            )
        
     
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    
    # t_list[i] * clip_norm / max(global_norm, clip_norm)
    # where:
    # global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
    
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step = global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling
    sample_input = tf.placeholder(tf.float32, shape = [1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
        )
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
        
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                 saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [12]:
num_steps = 7001
summary_frequency = 100

tic = time.clock()

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('initilized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        
        _, batch_loss, predictions, batch_learning_rate = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict = feed_dict
        )
        mean_loss += batch_loss
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                #the mean los is an estimate of the loss over the last few batches
            print(
                "Average loss at step %d: %f learning rate: %.2f" % (step, mean_loss, batch_learning_rate)
            )
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minimbatch perplexity: %2f' % float(np.exp(logprob(predictions, labels))))
            
            if step % (summary_frequency * 10) == 0:
                # Generate some samples
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            
            # Measure validation set perplexity
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
            
toc  = time.clock()

print(toc-tic)

initilized
Average loss at step 0: 3.293460 learning rate: 10.00
Minimbatch perplexity: 26.935911
vxaehfc ad  inqhlrasiqodkehqezvi bgnlnkhewcrdap b ya iihbev  aob r   mzkfeoxj ze
lldrwd t oe q   oyutuwijn rhecmo eyxfenezoe  eirsoe xse itgisopp ttreruchee ab a
rsujae eikhoj fdmtttlwdeetm g nnj ftswtyc xeqq otqdtenrf sa hzq swiirpsllxueeye 
dngughqdfmpjfeao f xvuyznsw  tqpjg fx wvki cmaomkterwpok  heziwvbsht enzi dexuqr
dzdueyfqvme jqar  r  eo end skw stv bvkv fjkn edsmrr  e wygli eoz tqnnmjbemf mp 
Validation set perplexity: 20.00
Average loss at step 100: 2.578932 learning rate: 10.00
Minimbatch perplexity: 10.635370
Validation set perplexity: 10.79
Average loss at step 200: 2.252436 learning rate: 10.00
Minimbatch perplexity: 8.310765
Validation set perplexity: 8.66
Average loss at step 300: 2.083221 learning rate: 10.00
Minimbatch perplexity: 6.357416
Validation set perplexity: 7.95
Average loss at step 400: 2.032548 learning rate: 10.00
Minimbatch perplexity: 7.836066
Validation set

### Problem 2

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

#### a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

In [35]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
    
    
    # 일반적인 표기법과 달리
    # i => x,
    # h => ouput
    # c=> state
       
    
    # Parameters:
    
    # vocabulary_lookup_embedding
    vocabulary_embedding = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    
    
    
    # Input gate: input, previous ouput, and bias
    ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Forget gate: input, previous output, and bias
    fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Memory cell: input, state, and bias
    cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Output gate: input, previous output, and bias
    ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Concantate parameters into single matrix
    conc_x = tf.concat(1, [ix, fx, cx, ox]) # => conc_x = [ix, fx, cx, ox]
    conc_m = tf.concat(1, [im, fm, cm, om]) # => conc_m = [im, fm, cm, om]
    conc_b = tf.concat(1, [ib, fb, cb, ob]) # => conc_b = [ib, fb, cb, ob]
    
    # Variables saving state across unrollings
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable = False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable = False)
    
    # Classifier weights and biases
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    
    """
    LSTM ARCHITECTURES
    
    i_t       = sigmoid (W_i(h_t-1, x_t) + b_i)
    f_t       = sigmoid (W_f(h_t-1, x_t) + b_f)
    o_t       = sigmoid (W_o(h_t-1, x_t) + b_o)
    
    
    c_tilda_t = tanh    (Wi(h_t-1, x_t)  + b_c)
    
    c_t       = f_t * c_t-1    +    i_t * c_tilda_t
    
    
    h_t       = o_t * tanh(c_t)

    """
    
    
    def lstm_cell(i, o, state):
        """Create a LSTM cell"""
        
        conc_mat = tf.matmul(i, conc_x) + tf.matmul(o, conc_m) + conc_b
        conc_input, conc_forget, update, conc_output = tf.split(1,4, conc_mat)
        
        input_gate = tf.sigmoid(conc_input)
        forget_gate = tf.sigmoid(conc_forget)
        output_gate = tf.sigmoid(conc_output)
        

        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    # Input data
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.float32, shape = [batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings] # (10 ~)
    train_labels = train_data[1:]
    
    # Unrolled LSTM loop
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:

        
        i_embed = tf.nn.embedding_lookup(vocabulary_embedding, tf.argmax(i, dimension=1))
        # tf.nn.embedding_lookup(params, ids) => train_input의 각 문자에서 가장 큰 값에 대응되는 vocabulary_embedding의 알파벳을 출력
        
        output, state = lstm_cell(i_embed, output, state)
        outputs.append(output)
    
    # State saving across unrollings
    with tf.control_dependencies([saved_output.assign(output),
                                 saved_state.assign(state)]):
    
        """
        with g.control_dependencies([a, b, c]):
              `d` and `e` will only run after `a`, `b`, and `c` have executed.
          d = ...
          e = ...
        """
    
        # Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels))
            )
        
     
    # Optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    
    # t_list[i] * clip_norm / max(global_norm, clip_norm)
    # where:
    # global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
    
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step = global_step)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling
    sample_input = tf.placeholder(tf.float32, shape = [1, vocabulary_size])
    sample_input_embedding = tf.nn.embedding_lookup(vocabulary_embedding, tf.argmax(sample_input, dimension=1))
    
    
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
        )
    
    
    sample_output, sample_state = lstm_cell(
        sample_input_embedding, saved_sample_output, saved_sample_state)
        
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                 saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [36]:
num_steps = 7001
summary_frequency = 100

tic = time.clock()

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('initilized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        
        _, batch_loss, predictions, batch_learning_rate = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict = feed_dict
        )
        mean_loss += batch_loss
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                #the mean los is an estimate of the loss over the last few batches
            print(
                "Average loss at step %d: %f learning rate: %.2f" % (step, mean_loss, batch_learning_rate)
            )
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minimbatch perplexity: %2f' % float(np.exp(logprob(predictions, labels))))
            
            if step % (summary_frequency * 10) == 0:
                # Generate some samples
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            
            # Measure validation set perplexity
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
            
toc  = time.clock()

print(toc-tic)

initilized
Average loss at step 0: 3.327611 learning rate: 10.00
Minimbatch perplexity: 27.871697
uovcjavt t wrk f hwweo n cize   et pn fmqogxcefoii hemqdgjo ohfop hau wgmztiawlk
vw radp vh mikeqx qruex vln afbinafanht vyr go  ertat cafwedaeyrdad eeefzzs zvk 
srydh st o l n ubfee u ydbliwamviifaxsdob rt hneyji eopya eu ttejmceqjm oa  cl a
ckivo  blebim zchi dadefn ma u afbben  hd ne circbjmus fvacpersggeg k meltgiicbi
uxas  ri   oapkk mp  tfeyftidfroriilfet eivs o ec agyk n eh ijyk g th  l amtogiq
Validation set perplexity: 20.60
Average loss at step 100: 2.291780 learning rate: 10.00
Minimbatch perplexity: 8.422700
Validation set perplexity: 8.53
Average loss at step 200: 1.996780 learning rate: 10.00
Minimbatch perplexity: 7.020111
Validation set perplexity: 6.89
Average loss at step 300: 1.917580 learning rate: 10.00
Minimbatch perplexity: 6.654817
Validation set perplexity: 6.55
Average loss at step 400: 1.854388 learning rate: 10.00
Minimbatch perplexity: 7.247853
Validation set p

#### b- Write a bigram-based LSTM, modeled on the character LSTM above.

#### c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this article. http://arxiv.org/abs/1409.2329

### Problem 3

### Problem 4